In [ ]:
!gdown --id 1Y3ACTjJCPvYaNTTHRXJRQBqAt5wvDLjq

In [ ]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
from google.colab.patches import cv2_imshow
def fit_factor(dim, dis):
    fac = round(dis / dim, 2)
    return fac

def map_factor(img_dim, s2, tar_dim):
    if tar_dim[1] > img_dim[0] - (s2 + 80):
        new_dim = img_dim[0] - (s2 + 40)
    else:
        new_dim = tar_dim[1]
    return new_dim

def acc_crop_side(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    return extTop

def acc_crop_bottom(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, mask = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    return extBot, extRight, extLeft

def acc_cor_points(oimage):
    first_half = oimage[0:oimage.shape[1], 0:int(oimage.shape[0] / 2)]
    second_half = oimage[0:oimage.shape[1], int(oimage.shape[0] / 2):oimage.shape[0]]
    topl = acc_crop_side(first_half)
    topr = acc_crop_side(second_half)
    bot, right, left = acc_crop_bottom(oimage)
    dis = (int(oimage.shape[0] / 2) + topr[0])
    if topl[1] > topr[1]:
        y = topr[1]
    else:
        y = topl[1]
    if topl[0] > left[0]:
        x = left[0]
    else:
        x = topl[0]
    if dis > right[0]:
        w = dis
    else:
        w = right[0]
    h = bot[1]
    return x, y, w, h

# Load the image
oimage = cv2.imread('/content/chain.png')  # Replace 'neclace4.jpg' with the path to your image file
print(oimage.shape)
x, y, w, h = acc_cor_points(oimage)
print(h, y, x, w)
image1 = oimage[y:h, x:w]

# Load the face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

# Process the image
frame = cv2.imread('/content/thalaindia.jpg')  # Replace 'your_image.jpg' with the path to your image file

# Get the frame dimensions
height, width, _ = frame.shape

# Detect faces in the frame
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)

# Loop over the faces
for (i, rect) in enumerate(rects):
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # Get the relevant facial landmarks for necklace placement
    (s1, s2) = shape[3]  # Example landmarks, adjust as needed
    (e1, e2) = shape[13]  # Example landmarks, adjust as needed
    dis = e1 - s1
    dim = image1.shape[1]

    # Calculate factors and dimensions for necklace placement
    m_factor = fit_factor(dim, dis)
    p_dim = (int(image1.shape[1] * m_factor), int(image1.shape[0] * m_factor))
    mf = map_factor(frame.shape, s2, p_dim)
    if mf == p_dim[0]:
        dim = p_dim
    else:
        dim = (p_dim[0], mf)

    # Resize and process the necklace image
    resized = cv2.resize(image1, dim, interpolation=cv2.INTER_AREA)
    img2gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
    mask_inv = cv2.bitwise_not(mask)
    resized1 = cv2.resize(resized, (resized.shape[1] * 10, resized.shape[0] * 10), interpolation=cv2.INTER_AREA)
    mask = cv2.resize(mask, (resized.shape[1] * 10, resized.shape[0] * 10), interpolation=cv2.INTER_AREA)
    mask_inv = cv2.resize(mask_inv, (resized.shape[1] * 10, resized.shape[0] * 10), interpolation=cv2.INTER_AREA)

    # Extract the region of interest for necklace placement
    point1 = s2 + 22 # Adjust as needed
    roi1 = frame[point1:point1 + dim[1], s1:s1 + dim[0]]
    roi = cv2.resize(roi1, (roi1.shape[1] * 10, roi1.shape[0] * 10), interpolation=cv2.INTER_AREA)

    # Mask and overlay the necklace on the frame
    roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
    roi_fg = cv2.bitwise_and(resized1, resized1, mask=mask)
    dst1 = cv2.add(roi_bg, roi_fg)
    dst = cv2.resize(dst1, (int(dst1.shape[1] / 10), int(dst1.shape[0] / 10)), interpolation=cv2.INTER_AREA)

    # Place the necklace on the frame
    frame[point1:point1 + dim[1], s1:s1 + dim[0]] = dst

# Display the resulting frame
cv2_imshow(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()